<a href="https://colab.research.google.com/github/fpcarneiro/dog-breed-identification/blob/master/Dog_Breed_Identification%20-%20Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip uninstall tensorflow
#!pip install tensorflow==2.0.0

Uninstalling tensorflow-1.15.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/freeze_graph
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-1.15.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-1.15.0


In [5]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"fpcarneiro","key":"a1d45a46756f42ebbcc45e148f67b615"}'}

In [0]:
!pip install -q kaggle

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [9]:
!kaggle datasets list

ref                                                      title                                               size  lastUpdated          downloadCount  
-------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  
chirin/africa-economic-banking-and-systemic-crisis-data  Africa Economic, Banking and Systemic Crisis Data   14KB  2019-07-21 02:00:17           5490  
tristan581/17k-apple-app-store-strategy-games            17K Mobile Strategy Games                            8MB  2019-08-26 08:22:16          12734  
gustavomodelli/forest-fires-in-brazil                    Forest Fires in Brazil                              31KB  2019-08-24 16:09:16          15054  
akhilv11/border-crossing-entry-data                      Border Crossing Entry Data                           4MB  2019-08-21 14:51:34           6263  
ruslankl/european-union-lgbt-survey-2012                 EU LGBT Survey                 

In [0]:
import pandas as pd
import numpy as np
import scipy
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import sys
from PIL import Image
import IPython.display as display
sys.modules['Image'] = Image

In [0]:
def show_batch(image_batch, label_batch):
    plt.figure(figsize=(15,15))
    for n in range(25):
        ax = plt.subplot(5,5,n+1)
        plt.imshow(image_batch[n])
        #plt.title(label_batch[n].shape)
        plt.title(CLASS_NAMES[label_batch[n]==1][0])
        plt.axis('off')

In [0]:
BATCH_SIZE = 32
IMG_HEIGHT = 300
IMG_WIDTH = 300

In [4]:
labels = pd.read_csv("input/labels.csv")
labels.id = labels.id + ".jpg"

FileNotFoundError: ignored

In [0]:
datagen = ImageDataGenerator(rescale = 1./255, validation_split=0.2)

In [0]:
training_generator = datagen.flow_from_dataframe(dataframe = labels, directory="input/train",
                                              x_col="id", y_col="breed", class_mode="categorical", 
                                              target_size=(IMG_HEIGHT, IMG_WIDTH), batch_size=BATCH_SIZE,
                                             subset="training")

validation_generator = datagen.flow_from_dataframe(dataframe = labels, directory="input/train",
                                              x_col="id", y_col="breed", class_mode="categorical", 
                                              target_size=(IMG_HEIGHT, IMG_WIDTH), batch_size=BATCH_SIZE,
                                             subset="validation")

Found 8178 validated image filenames belonging to 120 classes.
Found 2044 validated image filenames belonging to 120 classes.


In [0]:
training_generator.image_shape

(300, 300, 3)

In [0]:
CLASS_NAMES = np.array(list(training_generator.class_indices.keys()))
INPUT_SHAPE = training_generator.image_shape

In [0]:
#image_batch, label_batch = next(train_generator)
#show_batch(image_batch, label_batch)

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, Dropout, Flatten, Dense, BatchNormalization

model = Sequential()
model.add(Conv2D(filters=32, kernel_size=3, padding='same', input_shape=INPUT_SHAPE))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Conv2D(filters=64, kernel_size=3, padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Conv2D(filters=128, kernel_size=2, padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Flatten())
#model.add(Dense(64, activation='softmax'))
model.add(Dense(CLASS_NAMES.shape[0], activation='softmax'))

model.summary()
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 300, 300, 32)      896       
_________________________________________________________________
activation_3 (Activation)    (None, 300, 300, 32)      0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 150, 150, 32)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 150, 150, 64)      18496     
_________________________________________________________________
activation_4 (Activation)    (None, 150, 150, 64)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 75, 75, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 75, 75, 128)      

In [0]:
history = model.fit_generator(training_generator,
                              epochs=10,
                              verbose=1,
                              validation_data=validation_generator)

Epoch 1/10
  7/256 [..............................] - ETA: 21:01 - loss: 11.0246 - accuracy: 0.0045